<a href="https://colab.research.google.com/github/SarangGami/IPL-first-innings-score-prediction/blob/main/IPL_first_innings_score_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data manipulation libraries
import pandas as pd
import numpy as np

# Data visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import seaborn as sns

# Preprocessing libraries
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

# For build pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline


# Machine learning models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.ensemble import VotingRegressor,StackingRegressor


# for plot decision tree
from sklearn import tree

# Model selection libraries
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

# importing XGB regressor
from xgboost import XGBRegressor

# Metrics libraries for model evaluation
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

# Warnings module handles warnings in Python
import warnings
warnings.filterwarnings('ignore')

### **Data Gathering & accessing**

In [ ]:
# Loading the dataset
df = pd.read_csv('/content/drive/MyDrive/Cohort Durban Almabetter (DS)/IPL Project/ipl.csv')

In [ ]:
df.head()

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
0,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,1,0,0.1,1,0,0,0,222
1,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,1,0,0.2,1,0,0,0,222
2,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.2,2,0,0,0,222
3,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.3,2,0,0,0,222
4,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.4,2,0,0,0,222


In [ ]:
df.tail()

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
76009,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,DT Christian,121,7,19.2,40,0,40,12,129
76010,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,DT Christian,127,7,19.3,46,0,46,12,129
76011,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,DT Christian,128,7,19.4,47,0,47,12,129
76012,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,MG Johnson,DT Christian,129,7,19.5,48,0,47,13,129
76013,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,KH Pandya,DT Christian,129,8,19.6,47,1,47,13,129


In [ ]:
df.shape

(76014, 15)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76014 entries, 0 to 76013
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   mid             76014 non-null  int64  
 1   date            76014 non-null  object 
 2   venue           76014 non-null  object 
 3   bat_team        76014 non-null  object 
 4   bowl_team       76014 non-null  object 
 5   batsman         76014 non-null  object 
 6   bowler          76014 non-null  object 
 7   runs            76014 non-null  int64  
 8   wickets         76014 non-null  int64  
 9   overs           76014 non-null  float64
 10  runs_last_5     76014 non-null  int64  
 11  wickets_last_5  76014 non-null  int64  
 12  striker         76014 non-null  int64  
 13  non-striker     76014 non-null  int64  
 14  total           76014 non-null  int64  
dtypes: float64(1), int64(8), object(6)
memory usage: 8.7+ MB


In [ ]:
df.describe()

,mid,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
count,76014.000000,76014.000000,76014.000000,76014.000000,76014.000000,76014.000000,76014.000000,76014.000000,76014.000000
mean,308.627740,74.889349,2.415844,9.783068,33.216434,1.120307,24.962283,8.869287,160.901452
std,178.156878,48.823327,2.015207,5.772587,14.914174,1.053343,20.079752,10.795742,29.246231
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67.000000
25%,154.000000,34.000000,1.000000,4.600000,24.000000,0.000000,10.000000,1.000000,142.000000
50%,308.000000,70.000000,2.000000,9.600000,34.000000,1.000000,20.000000,5.000000,162.000000
75%,463.000000,111.000000,4.000000,14.600000,43.000000,2.000000,35.000000,13.000000,181.000000
max,617.000000,263.000000,10.000000,19.600000,113.000000,7.000000,175.000000,109.000000,263.000000


In [ ]:
df.duplicated().sum()

0

In [ ]:
df.isnull().sum()

mid               0
date              0
venue             0
bat_team          0
bowl_team         0
batsman           0
bowler            0
runs              0
wickets           0
overs             0
runs_last_5       0
wickets_last_5    0
striker           0
non-striker       0
total             0
dtype: int64

### **Data Cleaning**

In [ ]:
ipl_df = df.copy()

In [ ]:
ipl_df.sample(5)

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
64504,524,2016-04-25,"Punjab Cricket Association IS Bindra Stadium, ...",Mumbai Indians,Kings XI Punjab,JC Buttler,AR Patel,139,2,14.5,62,1,69,1,189
75919,617,2017-05-21,"Rajiv Gandhi International Stadium, Uppal",Mumbai Indians,Rising Pune Supergiant,AT Rayudu,SN Thakur,16,2,4.6,16,2,4,4,129
23618,192,2011-04-22,Wankhede Stadium,Mumbai Indians,Chennai Super Kings,SR Tendulkar,DE Bollinger,0,0,1.2,0,0,0,0,164
21037,170,2010-04-25,Dr DY Patil Sports Academy,Chennai Super Kings,Mumbai Indians,SK Raina,KA Pollard,111,3,15.2,47,1,30,20,168
50360,409,2014-04-30,Dubai International Cricket Stadium,Sunrisers Hyderabad,Mumbai Indians,DA Warner,Harbhajan Singh,73,2,10.4,31,0,25,17,172


In [ ]:
# Removing unwanted columns

columns_to_remove = ['mid', 'venue', 'batsman', 'bowler', 'striker', 'non-striker']
ipl_df.drop(labels=columns_to_remove, axis=1, inplace=True)

In [ ]:
ipl_df.sample(5)

,date,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total
58269,2015-04-26,Delhi Daredevils,Royal Challengers Bangalore,40,4,6.5,29,3,95
17157,2010-03-31,Delhi Daredevils,Rajasthan Royals,86,4,10.5,36,1,188
3926,2008-05-11,Kolkata Knight Riders,Deccan Chargers,19,1,3.1,19,1,204
31555,2012-04-14,Chennai Super Kings,Pune Warriors,71,2,9.5,32,2,155
41102,2013-04-16,Delhi Daredevils,Royal Challengers Bangalore,118,4,17.4,37,1,152


In [ ]:
ipl_df['bat_team'].unique()

array(['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
       'Mumbai Indians', 'Deccan Chargers', 'Kings XI Punjab',
       'Royal Challengers Bangalore', 'Delhi Daredevils',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant'], dtype=object)

In [ ]:
# Keeping only consistent teams
consistent_teams = ['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
                    'Mumbai Indians', 'Kings XI Punjab', 'Royal Challengers Bangalore',
                    'Delhi Daredevils', 'Sunrisers Hyderabad']

In [ ]:
ipl_df = ipl_df[(ipl_df['bat_team'].isin(consistent_teams)) & (ipl_df['bowl_team'].isin(consistent_teams))]

In [ ]:
# Removing the first 5 overs data in every match
ipl_df = ipl_df[ipl_df['overs']>=5.0]

In [ ]:
ipl_df.shape

(40108, 9)

In [ ]:
# Converting the column 'date' from string into datetime object

ipl_df['date']= pd.to_datetime(ipl_df['date'])

In [ ]:
ipl_df['crr'] =ipl_df['runs']/ipl_df['overs']

In [ ]:
# Split the 'overs' column into its integer and decimal parts
overs_int = ipl_df['overs'].apply(lambda x: int(x))
overs_dec = ipl_df['overs'].apply(lambda x: float('0.' + str(x).split('.')[1]))

# Calculate the number of balls left in each over
balls_left = (20 - overs_int - 1) * 6 + (6 - ((overs_dec * 10) % 10 + 1))

# Add the 'balls left' column to the DataFrame
ipl_df['balls_left'] = balls_left.astype(int)

In [ ]:
ipl_df['wickets_left'] = 10-ipl_df['wickets']

In [ ]:
ipl_df

,date,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total,crr,balls_left,wickets_left
32,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,0,5.1,59,0,222,11.960784,88,10
33,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.2,59,1,222,11.730769,87,9
34,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.3,59,1,222,11.509434,86,9
35,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.4,59,1,222,11.296296,85,9
36,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.5,58,1,222,11.090909,84,9
...,...,...,...,...,...,...,...,...,...,...,...,...
75884,2017-05-19,Kolkata Knight Riders,Mumbai Indians,106,9,18.1,29,4,107,5.856354,10,1
75885,2017-05-19,Kolkata Knight Riders,Mumbai Indians,107,9,18.2,29,4,107,5.879121,9,1
75886,2017-05-19,Kolkata Knight Riders,Mumbai Indians,107,9,18.3,28,4,107,5.846995,8,1
75887,2017-05-19,Kolkata Knight Riders,Mumbai Indians,107,9,18.4,24,4,107,5.815217,7,1


In [ ]:
ipl_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40108 entries, 32 to 75888
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            40108 non-null  datetime64[ns]
 1   bat_team        40108 non-null  object        
 2   bowl_team       40108 non-null  object        
 3   runs            40108 non-null  int64         
 4   wickets         40108 non-null  int64         
 5   overs           40108 non-null  float64       
 6   runs_last_5     40108 non-null  int64         
 7   wickets_last_5  40108 non-null  int64         
 8   total           40108 non-null  int64         
 9   crr             40108 non-null  float64       
 10  balls_left      40108 non-null  int64         
 11  wickets_left    40108 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(7), object(2)
memory usage: 4.0+ MB


## **pre-processing and model implementation**

In [ ]:
ipl_df.head()

,date,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total,crr,balls_left,wickets_left
32,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,0,5.1,59,0,222,11.960784,88,10
33,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.2,59,1,222,11.730769,87,9
34,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.3,59,1,222,11.509434,86,9
35,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.4,59,1,222,11.296296,85,9
36,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.5,58,1,222,11.090909,84,9


In [ ]:
# Splitting the data into train and test set

X_train = ipl_df.drop(labels='total', axis=1)[ipl_df['date'].dt.year <= 2015]
X_test = ipl_df.drop(labels='total', axis=1)[ipl_df['date'].dt.year >= 2016]
y_train = ipl_df[ipl_df['date'].dt.year <= 2015]['total'].values
y_test = ipl_df[ipl_df['date'].dt.year >= 2016]['total'].values

In [ ]:
# Removing the 'date' column

X_train.drop(labels='date', axis=1, inplace=True)
X_test.drop(labels='date', axis=1, inplace=True)

In [ ]:
# split sizes
print(X_train.shape)
print(X_test.shape)

(34464, 10)
(5644, 10)


In [ ]:
X_train.head()

,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,crr,balls_left,wickets_left
32,Kolkata Knight Riders,Royal Challengers Bangalore,61,0,5.1,59,0,11.960784,88,10
33,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.2,59,1,11.730769,87,9
34,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.3,59,1,11.509434,86,9
35,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.4,59,1,11.296296,85,9
36,Kolkata Knight Riders,Royal Challengers Bangalore,61,1,5.5,58,1,11.090909,84,9


In [ ]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['bat_team','bowl_team'])
]
,remainder='passthrough')

In [ ]:
pipe1 = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',LinearRegression())
])


# fit the pipeline on training dataset
pipe1.fit(X_train,y_train)

# predict the train and test dataset 
y_pred_train = pipe1.predict(X_train)
y_pred = pipe1.predict(X_test)

# LinearRegression model all output scores
print('\033[1mTraining data R2 and Adjusted R2 Score\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('R2 score',r2_score(y_train,y_pred_train))
print('Adjusted R2 score', (1-(1-r2_score(y_train,y_pred_train))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1))))

print('\n')
print('\033[1mTesting data R2 and Adjusted R2 Score\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('R2 score',r2_score(y_test,y_pred))
print('Adjusted R2 score', (1-(1-r2_score(y_test,y_pred))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))))

print('\n')
print('\033[1mThe performance metrics\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('MAE',mean_absolute_error(y_test,y_pred))
print('MSE',mean_squared_error(y_test,y_pred))
print('RMSE',np.sqrt(mean_squared_error(y_test,y_pred)))

Training data R2 and Adjusted R2 Score
-----------------------------------------
R2 score 0.6531004593685696
Adjusted R2 score 0.6529997716082494


Testing data R2 and Adjusted R2 Score
-----------------------------------------
R2 score 0.7234947055136165
Adjusted R2 score 0.7230038386673776


The performance metrics
-----------------------------------------
MAE 11.905133946918916
MSE 246.20988344424788
RMSE 15.691076554661501


In [ ]:
# apply DecisionTreeRegressor algorithm with hyperparameter tuning

# giving parameters
parameters = {
    'criterion':['squared_error'],                    # 'friedman_mse', 'absolute_error'
    'splitter' :['best'],                             # random
    'max_depth' :[7],                                 # 4,5,6,7,8,9,None
    'max_features' :[1.0]                             # 0.25,0.50,0.75,0.85
}

# we use gridsearchCV because the dataset is not that big
dtr = GridSearchCV(DecisionTreeRegressor(), param_grid=parameters , cv=5, n_jobs=-1)


pipe2 = Pipeline(steps=[
    ('step1',trf),
    ('step3',dtr)
])


# fit the pipeline on training dataset
pipe2.fit(X_train,y_train)

# predict the train and test dataset 
y_pred_train = pipe2.predict(X_train)
y_pred = pipe2.predict(X_test)


# DecisionTreeRegressor model all output scores
print('\033[1mTraining data R2 and Adjusted R2 Score\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('R2 score',r2_score(y_train,y_pred_train))
print('Adjusted R2 score', (1-(1-r2_score(y_train,y_pred_train))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1))))

print('\n')
print('\033[1mTesting data R2 and Adjusted R2 Score\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('R2 score',r2_score(y_test,y_pred))
print('Adjusted R2 score', (1-(1-r2_score(y_test,y_pred))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))))

print('\n')
print('\033[1mCross-validation score and best params\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print("The best parameters is", dtr.best_params_)
print('cross-validation score', dtr.best_score_)

print('\n')
print('\033[1mThe performance metrics\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('MAE',mean_absolute_error(y_test,y_pred))
print('MSE',mean_squared_error(y_test,y_pred))
print('RMSE',np.sqrt(mean_squared_error(y_test,y_pred)))

Training data R2 and Adjusted R2 Score
-----------------------------------------
R2 score 0.7131034493800334
Adjusted R2 score 0.7130201775167356


Testing data R2 and Adjusted R2 Score
-----------------------------------------
R2 score 0.5844868659769176
Adjusted R2 score 0.5837492250501946


Cross-validation score and best params
-----------------------------------------
The best parameters is {'criterion': 'squared_error', 'max_depth': 7, 'max_features': 1.0, 'splitter': 'best'}
cross-validation score 0.50544964417623


The performance metrics
-----------------------------------------
MAE 13.53631633600114
MSE 369.987274520037
RMSE 19.235053275726507


In [ ]:
# apply RandomForestRegressor algorithm with hyperparameter tuning


# giving parameters
parameters = {
    'n_estimators':[65],                        # 50,55,60,70,80,90,100
    'max_depth' :[10],                          # 4,5,6,7,8,9,None
    #'max_features' :['sqrt','log2',None]       # 'sqrt','log2'
    'max_samples' :[0.1]                        # 0.40,0.50,0.60,0.70,0.75,0.85,1.0
}
 
# we use gridsearchCV because the dataset is not that big so we use this not RandomizedSearchCV
rfr = GridSearchCV(RandomForestRegressor(), param_grid=parameters , cv=5, n_jobs=-1)

pipe3 = Pipeline(steps=[
    ('step1',trf),
    ('step3',rfr)
])

# fit the pipeline on training dataset
pipe3.fit(X_train,y_train)

# predict the train and test dataset 
y_pred_train = pipe3.predict(X_train)
y_pred = pipe3.predict(X_test)

# RandomForestRegressor model all output scores
print('\033[1mTraining data R2 and Adjusted R2 Score\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('R2 score',r2_score(y_train,y_pred_train))
print('Adjusted R2 score', (1-(1-r2_score(y_train,y_pred_train))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1))))

print('\n')
print('\033[1mTesting data R2 and Adjusted R2 Score\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('R2 score',r2_score(y_test,y_pred))
print('Adjusted R2 score', (1-(1-r2_score(y_test,y_pred))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))))

print('\n')
print('\033[1mCross-validation score and best params\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print("The best parameters is", rfr.best_params_)
print('cross-validation score', rfr.best_score_)

print('\n')
print('\033[1mThe performance metrics\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('MAE',mean_absolute_error(y_test,y_pred))
print('MSE',mean_squared_error(y_test,y_pred))
print('RMSE',np.sqrt(mean_squared_error(y_test,y_pred)))

Training data R2 and Adjusted R2 Score
-----------------------------------------
R2 score 0.8032028218063052
Adjusted R2 score 0.8031457013296577


Testing data R2 and Adjusted R2 Score
-----------------------------------------
R2 score 0.7087963770554606
Adjusted R2 score 0.7082794169579202


Cross-validation score and best params
-----------------------------------------
The best parameters is {'max_depth': 10, 'max_samples': 0.1, 'n_estimators': 65}
cross-validation score 0.5971294505721503


The performance metrics
-----------------------------------------
MAE 12.068433542096194
MSE 259.2977837798634
RMSE 16.1027259735693


In [ ]:
# apply XGBRegressor algorithm with hyperparameter tuning


# giving parameters
parameters = {
    'n_estimators' :[150],                                                    # 150,160,170,180,200
    'learning_rate' :[0.05],                                                  # 0.001,0.005,0.01,0.05,0.1
    'max_depth' :[4]                                                          # 3,4,5,6,7,8
}
 
# we use gridsearchCV because the dataset is not that big so we use this not RandomizedSearchCV
xgbr = GridSearchCV(XGBRegressor(objective='reg:squarederror'), param_grid=parameters , cv=5, n_jobs=-1)

pipe5 = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',xgbr)
])

# fit the pipeline on training dataset
pipe5.fit(X_train,y_train)

# predict the train and test dataset 
y_pred_train = pipe5.predict(X_train)
y_pred = pipe5.predict(X_test)

# GradientBoostingRegressor model all output scores
print('\033[1mTraining data R2 and Adjusted R2 Score\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('R2 score',r2_score(y_train,y_pred_train))
print('Adjusted R2 score', (1-(1-r2_score(y_train,y_pred_train))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1))))

print('\n')
print('\033[1mTesting data R2 and Adjusted R2 Score\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('R2 score',r2_score(y_test,y_pred))
print('Adjusted R2 score', (1-(1-r2_score(y_test,y_pred))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))))

print('\n')
print('\033[1mCross-validation score and best params\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print("The best parameters is", xgbr.best_params_)
print('cross-validation score', xgbr.best_score_)

print('\n')
print('\033[1mThe performance metrics\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('MAE',mean_absolute_error(y_test,y_pred))
print('MSE',mean_squared_error(y_test,y_pred))
print('RMSE',np.sqrt(mean_squared_error(y_test,y_pred)))

Training data R2 and Adjusted R2 Score
-----------------------------------------
R2 score 0.745332859801201
Adjusted R2 score 0.745258942539947


Testing data R2 and Adjusted R2 Score
-----------------------------------------
R2 score 0.7296873813751007
Adjusted R2 score 0.729207508095099


Cross-validation score and best params
-----------------------------------------
The best parameters is {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 150}
cross-validation score 0.6012491293679119


The performance metrics
-----------------------------------------
MAE 11.97340346700667
MSE 240.69571054243704
RMSE 15.514371097225856


In [ ]:
# apply GradientBoostingRegressor algorithm with hyperparameter tuning as step2


# giving parameters
parameters = {
    'n_estimators' :[240],                                     # 60,100,140,160,180,200,220
    'learning_rate' :[0.05],                                       # 0.001,0.01,0.05,0.1,1.0
    'max_depth' : [8],                                                          # 4,5,6,7,8
    'max_leaf_nodes' :[8]                                     # 4,6,8,12,16,20,24,32,None        # 'loss' :'squared_error', 'absolute_error', 'huber'
}
 
# we use gridsearchCV because the dataset is not that big so we use this not RandomizedSearchCV
gbr = GridSearchCV(GradientBoostingRegressor(), param_grid=parameters , cv=5, n_jobs=-1)

pipe4 = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',gbr)
])

# fit the pipeline on training dataset
pipe4.fit(X_train,y_train)

# predict the train and test dataset 
y_pred_train = pipe4.predict(X_train)
y_pred = pipe4.predict(X_test)

# GradientBoostingRegressor model all output scores
print('\033[1mTraining data R2 and Adjusted R2 Score\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('R2 score',r2_score(y_train,y_pred_train))
print('Adjusted R2 score', (1-(1-r2_score(y_train,y_pred_train))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1))))

print('\n')
print('\033[1mTesting data R2 and Adjusted R2 Score\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('R2 score',r2_score(y_test,y_pred))
print('Adjusted R2 score', (1-(1-r2_score(y_test,y_pred))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))))

print('\n')
print('\033[1mCross-validation score and best params\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print("The best parameters is", gbr.best_params_)
print('cross-validation score', gbr.best_score_)

print('\n')
print('\033[1mThe performance metrics\033[0m')
print('\033[1m' + '-----------------------------------------' + '\033[0m')
print('MAE',mean_absolute_error(y_test,y_pred))
print('MSE',mean_squared_error(y_test,y_pred))
print('RMSE',np.sqrt(mean_squared_error(y_test,y_pred)))

Training data R2 and Adjusted R2 Score
-----------------------------------------
R2 score 0.7422095122091216
Adjusted R2 score 0.7421346883947104


Testing data R2 and Adjusted R2 Score
-----------------------------------------
R2 score 0.7383060769111658
Adjusted R2 score 0.7378415039960426


Cross-validation score and best params
-----------------------------------------
The best parameters is {'learning_rate': 0.05, 'max_depth': 8, 'max_leaf_nodes': 8, 'n_estimators': 240}
cross-validation score 0.597856358600211


The performance metrics
-----------------------------------------
MAE 11.78882074790651
MSE 233.02132576323146
RMSE 15.265036055091107


In [ ]:
import pickle

pickle.dump(pipe4,open('pipe.pkl','wb'))